In [35]:
from sqlite3 import connect
import pandas as pd
import numpy as np

In [30]:
conn = connect(':memory:')

invoices = pd.read_csv("/Users/manikhossain/Downloads/Rivet-Coding-Challenge/invoices.csv")
invoices['invoice_date'] = pd.to_datetime(invoices['invoice_date'], format="%Y-%m-%d")

payments = pd.read_csv("/Users/manikhossain/Downloads/Rivet-Coding-Challenge/payments.csv")
payments['payment_date'] = pd.to_datetime(payments['payment_date'], format="%Y-%m-%d")

invoices.to_sql('invoices', conn)
payments.to_sql('payments', conn)
pd.read_sql('SELECT * FROM invoices', conn)
# pd.read_sql('SELECT * FROM payments', conn)

,index,customer_id,customer_name,invoice_id,invoice_amount,invoice_balance,invoice_date,payment_terms
0,0,1,Microsoft,X-1,1000,0,2021-01-01 00:00:00,Net 30
1,1,2,Apple,X-2,1500,0,2021-01-15 00:00:00,Net 45
2,2,3,Google,X-3,2000,0,2021-03-01 00:00:00,Net 30
3,3,1,Microsoft,X-4,1000,0,2021-04-01 00:00:00,Net 30
4,4,4,Netflix,X-5,3000,0,2021-05-01 00:00:00,Net 30
5,5,4,Netflix,X-6,3000,500,2021-06-01 00:00:00,Net 30
6,6,5,Meta,X-7,500,0,2021-06-15 00:00:00,Net 30
7,7,2,Apple,X-8,5000,2000,2021-07-01 00:00:00,Net 45
8,8,3,Google,X-9,2500,2500,2021-08-01 00:00:00,Net 30
9,9,1,Microsoft,X-10,1000,1000,2021-11-01 00:00:00,Net 60


In [66]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

percentileDF = invoices.groupby('customer_id').agg({'invoice_amount': [percentile(90), percentile(70)]})
percentileDF.to_sql('invoicePercentile', conn)
percentileDF = pd.read_sql('SELECT * FROM invoicePercentile', conn)
percentileDF.rename(columns={list(percentileDF)[1]:'pct_90', list(percentileDF)[2]:'pct_70'}, inplace=True)

percentileDF.to_sql('percentileTbl', conn)
pd.read_sql('SELECT * FROM percentileTbl', conn)

,index,customer_id,pct_90,pct_70
0,0,1,1000,1000
1,1,2,4650,3950
2,2,3,2450,2350
3,3,4,3000,3000
4,4,5,725,675


# #### processed-customers.csv

In [70]:
aggregatedCustomers = pd.read_sql("SELECT i.customer_id, i.customer_name, sum(i.invoice_amount) total_invoiced_amount, \
          count(*) total_invoice_count, sum(i.invoice_balance) unpaid_amount, \
          count( (CASE WHEN i.invoice_balance > 0.0 then 1 ELSE null END) ) unpaid_count,\
          min(i.invoice_date) first_invoice_date\
          FROM invoices i group by i.customer_id, i.customer_name", conn)

aggregatedCustomers.to_sql('aggregatedCustomersTbl', conn)
pd.read_sql('SELECT * FROM aggregatedCustomersTbl', conn)

,index,customer_id,customer_name,total_invoiced_amount,total_invoice_count,unpaid_amount,unpaid_count,first_invoice_date
0,0,1,Microsoft,3000,3,1000,1,2021-01-01 00:00:00
1,1,2,Apple,6500,2,2000,1,2021-01-15 00:00:00
2,2,3,Google,4500,2,2500,1,2021-03-01 00:00:00
3,3,4,Netflix,6000,2,500,1,2021-05-01 00:00:00
4,4,5,Meta,1250,2,750,1,2021-06-15 00:00:00


In [135]:
aggregatedDF = pd.read_sql("select b.*, i.invoice_amount first_invoice_amount from aggregatedCustomersTbl b \
                          left join invoices i \
                          on b.first_invoice_date =  i.invoice_date and b.customer_id = i.customer_id", conn)
aggregatedDF.to_sql('aggregatedTbl', conn, if_exists='replace')

latestPaymentInfoDF = pd.read_sql("with base as (select customer_id, max(payment_date) payment_date from payments group by customer_id)\
                              select b.*, p.payment_amount from base b \
                              inner join payments p \
                              on b.payment_date =  p.payment_date and  b.customer_id = p.customer_id", conn)
latestPaymentInfoDF.to_sql('latestPaymentInfoTbl', conn, if_exists='replace')

processedCustomerDF = pd.read_sql("select pc.customer_id,pc.customer_name, pc.total_invoiced_amount,\
                                   pc.total_invoice_count, pc.unpaid_amount, pc.unpaid_count, pc.first_invoice_date, pc.first_invoice_amount,\
                                   p.payment_date last_payment_date, p.payment_amount last_payment_amount,\
                                   CASE WHEN pc.total_invoiced_amount >= pt.pct_90 THEN 'High' \
                                     WHEN pc.total_invoiced_amount > pt.pct_70 and pc.total_invoiced_amount < pt.pct_90 THEN 'Medium' \
                                     ELSE 'Low' \
                                   END AS customer_segment\
                                from aggregatedTbl pc \
                                left join latestPaymentInfoTbl p on pc.customer_id = p.customer_id\
                                left join percentileTbl pt on pc.customer_id = pt.customer_id", conn)

processedCustomerDF.to_sql('processedCustomersTbl', conn, if_exists='replace')

In [136]:
pd.read_sql("select * from processedCustomersTbl", conn)

,index,customer_id,customer_name,total_invoiced_amount,total_invoice_count,unpaid_amount,unpaid_count,first_invoice_date,first_invoice_amount,last_payment_date,last_payment_amount,customer_segment
0,0,1,Microsoft,3000,3,1000,1,2021-01-01 00:00:00,1000,2021-06-01 00:00:00,1000,High
1,1,2,Apple,6500,2,2000,1,2021-01-15 00:00:00,1500,2021-08-15 00:00:00,3000,High
2,2,3,Google,4500,2,2500,1,2021-03-01 00:00:00,2000,2021-04-01 00:00:00,1000,High
3,3,4,Netflix,6000,2,500,1,2021-05-01 00:00:00,3000,2021-07-31 00:00:00,2500,High
4,4,5,Meta,1250,2,750,1,2021-06-15 00:00:00,500,2021-07-15 00:00:00,500,High


# #### processed-invoices.csv

In [137]:
populatedInvoiceDF = pd.read_sql("select *, \
          CASE WHEN payment_terms = 'Net 30' THEN DATE(invoice_date, '30 day') \
               WHEN payment_terms = 'Net 45' THEN DATE(invoice_date, '45 day') \
               WHEN payment_terms = 'Net 60' THEN DATE(invoice_date, '60 day') \
               ELSE invoice_date \
          END AS invoice_due_date \
          from invoices", conn)
populatedInvoiceDF.to_sql('populatedInvoiceTbl', conn, if_exists='replace')


processedInvoiceDF = pd.read_sql("select i.customer_id, i.customer_name, i.invoice_id, i.invoice_amount,i.invoice_balance, \
          i.invoice_date, i.payment_terms, i.invoice_due_date,\
          CASE WHEN date('now') > i.invoice_due_date  THEN 'true' ELSE 'false' \
          END AS invoice_overdue, pc.customer_segment\
          from populatedInvoiceTbl i left join processedCustomersTbl pc on i.customer_id = pc.customer_id", conn)

processedInvoiceDF.to_sql('processedInvoiceTbl', conn, if_exists='replace')


In [138]:
pd.read_sql("select * from processedInvoiceTbl", conn)

,index,customer_id,customer_name,invoice_id,invoice_amount,invoice_balance,invoice_date,payment_terms,invoice_due_date,invoice_overdue,customer_segment
0,0,1,Microsoft,X-1,1000,0,2021-01-01 00:00:00,Net 30,2021-01-31,true,High
1,1,2,Apple,X-2,1500,0,2021-01-15 00:00:00,Net 45,2021-03-01,true,High
2,2,3,Google,X-3,2000,0,2021-03-01 00:00:00,Net 30,2021-03-31,true,High
3,3,1,Microsoft,X-4,1000,0,2021-04-01 00:00:00,Net 30,2021-05-01,true,High
4,4,4,Netflix,X-5,3000,0,2021-05-01 00:00:00,Net 30,2021-05-31,true,High
5,5,4,Netflix,X-6,3000,500,2021-06-01 00:00:00,Net 30,2021-07-01,true,High
6,6,5,Meta,X-7,500,0,2021-06-15 00:00:00,Net 30,2021-07-15,true,High
7,7,2,Apple,X-8,5000,2000,2021-07-01 00:00:00,Net 45,2021-08-15,true,High
8,8,3,Google,X-9,2500,2500,2021-08-01 00:00:00,Net 30,2021-08-31,true,High
9,9,1,Microsoft,X-10,1000,1000,2021-11-01 00:00:00,Net 60,2021-12-31,false,High


# #### processed-payments.csv

In [139]:
processedPaymentDF = pd.read_sql("select p.customer_id, pc.customer_name, p.payment_id, p.payment_amount, p.payment_date, p.invoice_id, pc.customer_segment \
          from payments p left join processedCustomersTbl pc on p.customer_id = pc.customer_id", conn)
processedPaymentDF.to_sql('processedPaymentTbl', conn, if_exists='replace')
pd.read_sql("select * from processedPaymentTbl", conn)

,index,customer_id,customer_name,payment_id,payment_amount,payment_date,invoice_id,customer_segment
0,0,1,Microsoft,P-1,1000,2021-02-01 00:00:00,X-1,High
1,1,2,Apple,P-2,1500,2021-02-15 00:00:00,X-2,High
2,2,3,Google,P-3,1000,2021-03-15 00:00:00,X3,High
3,3,3,Google,P-4,1000,2021-04-01 00:00:00,X3,High
4,4,1,Microsoft,P-5,1000,2021-06-01 00:00:00,X-4,High
5,5,4,Netflix,P-6,3000,2021-06-01 00:00:00,X-5,High
6,6,4,Netflix,P-7,2500,2021-07-31 00:00:00,X-6,High
7,7,5,Meta,P-8,500,2021-07-15 00:00:00,X-7,High
8,8,2,Apple,P-9,3000,2021-08-15 00:00:00,X-8,High


In [140]:
processedPaymentDF.to_csv('processed-payment1.csv', index=False)
processedCustomerDF.to_csv('processed-customer1.csv', index=False)
processedInvoiceDF.to_csv('processed-invoice1.csv', index=False)